In [1]:
import requests
import pandas as pd
import time

TOKEN = "129ac2e3-e8a6-72c7-58c1-acced5a601bd"

# Diccionario de Indicadores de Población (ID -> [Rango de Edad, Sexo])
indicadores_poblacion = {
    # 0 a 4 años
    "1002000059": ("0 a 4 años", "Hombres"),
    "1002000060": ("0 a 4 años", "Mujeres"),
    # 5 a 9 años (Corregido de 4 a 9 para estándar quinquenal)
    "1002000089": ("5 a 9 años", "Hombres"),
    "1002000090": ("5 a 9 años", "Mujeres"),
    # 10 a 14 años
    "1002000062": ("10 a 14 años", "Hombres"),
    "1002000063": ("10 a 14 años", "Mujeres"),
    # 15 a 19 años
    "1002000068": ("15 a 19 años", "Hombres"),
    "1002000069": ("15 a 19 años", "Mujeres"),
    # 20 a 24 años
    "1002000071": ("20 a 24 años", "Hombres"),
    "1002000072": ("20 a 24 años", "Mujeres"),
    # 25 a 29 años
    "1002000074": ("25 a 29 años", "Hombres"),
    "1002000075": ("25 a 29 años", "Mujeres"),
    # 30 a 34 años
    "1002000077": ("30 a 34 años", "Hombres"),
    "1002000078": ("30 a 34 años", "Mujeres"),
    # 35 a 39 años
    "1002000080": ("35 a 39 años", "Hombres"),
    "1002000081": ("35 a 39 años", "Mujeres"),
    # 40 a 44 años
    "1002000083": ("40 a 44 años", "Hombres"),
    "1002000084": ("40 a 44 años", "Mujeres"),
    # 45 a 49 años
    "1002000086": ("45 a 49 años", "Hombres"),
    "1002000087": ("45 a 49 años", "Mujeres"),
    # 50 a 54 años
    "1002000092": ("50 a 54 años", "Hombres"),
    "1002000093": ("50 a 54 años", "Mujeres"),
    # 55 a 59 años
    "1002000095": ("55 a 59 años", "Hombres"),
    "1002000096": ("55 a 59 años", "Mujeres"),
    # 60 a 64 años
    "1002000098": ("60 a 64 años", "Hombres"),
    "1002000099": ("60 a 64 años", "Mujeres"),
    # 65 a 69 años
    "1002000101": ("65 a 69 años", "Hombres"),
    "1002000102": ("65 a 69 años", "Mujeres"),
    # 70 a 74 años
    "1002000104": ("70 a 74 años", "Hombres"),
    "1002000105": ("70 a 74 años", "Mujeres"),
    # 75 a 79 años
    "1002000107": ("75 a 79 años", "Hombres"),
    "1002000108": ("75 a 79 años", "Mujeres"),
    # 80 a 84 años
    "1002000110": ("80 a 84 años", "Hombres"),
    "1002000111": ("80 a 84 años", "Mujeres"),
    # 85 a 89 años
    "1002000113": ("85 a 89 años", "Hombres"),
    "1002000114": ("85 a 89 años", "Mujeres"),
    # 90 a 94 años
    "1002000116": ("90 a 94 años", "Hombres"),
    "1002000117": ("90 a 94 años", "Mujeres"),
    # 95 a 99 años
    "1002000119": ("95 a 99 años", "Hombres"),
    "1002000120": ("95 a 99 años", "Mujeres"),
    # 100 años y más
    "1002000065": ("100 años y más", "Hombres"),
    "1002000066": ("100 años y más", "Mujeres"),
}

resultados_totales = []
consultas_fallidas = []

def extraer_datos(ind_clave, rango, sexo, clave_estado, url):
    for intento in range(3):
        try:
            # Timeout de 15 segundos
            response = requests.get(url, timeout=15)
            
            if response.status_code == 200:
                data = response.json()
                if 'Series' in data and len(data['Series']) > 0:
                    serie = data['Series'][0]
                    if 'OBSERVATIONS' in serie:
                        for obs in serie['OBSERVATIONS']:
                            registro = {
                                'CLAVE_INDICADOR': ind_clave,
                                'RANGO_EDAD': rango,
                                'SEXO': sexo,
                                'CVE_ENT': obs.get('COBER_GEO', clave_estado),
                                'TIME_PERIOD': obs.get('TIME_PERIOD'),
                                'OBS_VALUE': obs.get('OBS_VALUE')
                            }
                            resultados_totales.append(registro)
                return True
            else:
                print(f"  ⚠️ Error HTTP {response.status_code} (Intento {intento+1}/3)")
                time.sleep(2)
        except Exception as e:
            print(f"  ⚠️ Fallo de red: {type(e).__name__} (Intento {intento+1}/3)")
            time.sleep(2)
    return False

print("--- INICIANDO EXTRACCIÓN DE POBLACIÓN (ÚLTIMO AÑO DISPONIBLE) ---")

# Bucle Principal
for ind_clave, (rango, sexo) in indicadores_poblacion.items():
    print(f"Consultando: {ind_clave} - {rango} ({sexo})")
    
    # Estados 00 a 32 (Incluye Nacional)
    for i in range(0, 33):
        clave_estado = f"{i:02d}"
        
        # URL AJUSTADA: "BISE" en lugar de "BIE-BISE" y "true" para último dato
        url = f"https://www.inegi.org.mx/app/api/indicadores/desarrolladores/jsonxml/INDICATOR/{ind_clave}/es/{clave_estado}/true/BISE/2.0/{TOKEN}?type=json"
        
        exito = extraer_datos(ind_clave, rango, sexo, clave_estado, url)
        
        if not exito:
            print(f"  ❌ Fallo total en {ind_clave}-{clave_estado}. Se envía a la cola final.")
            consultas_fallidas.append((ind_clave, rango, sexo, clave_estado, url))
            
        # Pausa ligera (al usar 'true' la respuesta es más ligera, podemos bajar un poco el tiempo)
        time.sleep(0.2)

# Reintentos de fallas
if consultas_fallidas:
    print("\n--- INICIANDO REINTENTOS DE CONSULTAS FALLIDAS ---")
    for ind_clave, rango, sexo, clave_estado, url in consultas_fallidas:
        print(f"Reintentando: {ind_clave} - Estado {clave_estado}")
        extraer_datos(ind_clave, rango, sexo, clave_estado, url)
        time.sleep(0.5)

# Procesamiento Final
if resultados_totales:
    df_poblacion = pd.DataFrame(resultados_totales)
    
    df_poblacion['OBS_VALUE'] = pd.to_numeric(df_poblacion['OBS_VALUE'], errors='coerce')
    
    # Ordenar para presentación limpia: Estado -> Rango Edad (Alfabético podría fallar, idealmente usar ID) -> Sexo
    # Ordenamos por ID de indicador que suele seguir el orden de edad
    df_poblacion = df_poblacion.sort_values(by=['CVE_ENT', 'CLAVE_INDICADOR', 'SEXO']).reset_index(drop=True)
    
    print("\n--- RESUMEN FINAL ---")
    print(f"Total de registros extraídos: {len(df_poblacion)}")
    print(df_poblacion.head(10))
    
    # Verificación de años obtenidos (Debería ser mayoritariamente un solo año reciente)
    print("\nAños encontrados en la extracción:")
    print(df_poblacion['TIME_PERIOD'].unique())
    
else:
    print("\nNo se encontraron datos.")

--- INICIANDO EXTRACCIÓN DE POBLACIÓN (ÚLTIMO AÑO DISPONIBLE) ---
Consultando: 1002000059 - 0 a 4 años (Hombres)
  ⚠️ Fallo de red: ConnectTimeout (Intento 1/3)
Consultando: 1002000060 - 0 a 4 años (Mujeres)
Consultando: 1002000089 - 5 a 9 años (Hombres)
Consultando: 1002000090 - 5 a 9 años (Mujeres)
  ⚠️ Fallo de red: ConnectTimeout (Intento 1/3)
Consultando: 1002000062 - 10 a 14 años (Hombres)
  ⚠️ Fallo de red: ConnectTimeout (Intento 1/3)
Consultando: 1002000063 - 10 a 14 años (Mujeres)
Consultando: 1002000068 - 15 a 19 años (Hombres)
Consultando: 1002000069 - 15 a 19 años (Mujeres)
Consultando: 1002000071 - 20 a 24 años (Hombres)
Consultando: 1002000072 - 20 a 24 años (Mujeres)
Consultando: 1002000074 - 25 a 29 años (Hombres)
Consultando: 1002000075 - 25 a 29 años (Mujeres)
  ⚠️ Fallo de red: ConnectTimeout (Intento 1/3)
Consultando: 1002000077 - 30 a 34 años (Hombres)
Consultando: 1002000078 - 30 a 34 años (Mujeres)
Consultando: 1002000080 - 35 a 39 años (Hombres)
  ⚠️ Fallo de r